In [1]:
"""Proyecto Puzzle-8 (Código Esqueleto)

**Link**
https://colab.research.google.com/drive/1oY87uNFG-MRFVnpwfaxnv5_0zMLCXUMm#scrollTo=e21PK-73C0nL

**Integrates del grupo**

Justin Jr. Minaya Gil (2019-8013)
Albert A Acosta P  (2019-7801)
Ricky Montero Terrero (2019-7944)
Willys Ambiorix Acevedo Montero (2019-7801)
Leonardo Ismael Peña Magallanez (2018-6923)
"""
# Librerías (Willys Acevedo)

import queue as Q
import time
import resource
import math
from time import time
import psutil
import os

def memory_usage_psutil():
    
    #return the memory usage in MB
    
    process = psutil.Process(os.getpid())
    mem = process.memory_info().rss / float(2 ** 20)
    return mem

#Puzzle Albert

class PuzzleState(object):

    """docstring para PuzzleState"""

    def __init__(self, config, n, parent=None, action="Initial", cost=0):

        if n*n != len(config) or n < 2:

            raise Exception("the length of config is not correct!")

        self.n = n

        self.cost = cost

        self.parent = parent

        self.action = action

        self.dimension = n

        self.config = config

        self.children = []

        for i, item in enumerate(self.config):

            if item == 0:

                self.blank_row = i // self.n

                self.blank_col = i % self.n

                break

    def display(self):

        for i in range(self.n):

            line = []

            offset = i * self.n

            for j in range(self.n):

                line.append(self.config[offset + j])

            print(line)

    def move_left(self):

        if self.blank_col == 0:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index - 1

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Left", cost=self.cost + 1)

    def move_right(self):

        if self.blank_col == self.n - 1:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index + 1

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Right", cost=self.cost + 1)

    def move_up(self):

        if self.blank_row == 0:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index - self.n

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Up", cost=self.cost + 1)

    def move_down(self):

        if self.blank_row == self.n - 1:

            return None

        else:

            blank_index = self.blank_row * self.n + self.blank_col

            target = blank_index + self.n

            new_config = list(self.config)

            new_config[blank_index], new_config[target] = new_config[target], new_config[blank_index]

            return PuzzleState(tuple(new_config), self.n, parent=self, action="Down", cost=self.cost + 1)

    def expand(self):

        """Expandir el nodo"""

        # Añadir nodos hijos en orden UDLR (Up-Down-Left-Right)

        if len(self.children) == 0:

            up_child = self.move_up()

            if up_child is not None:

                self.children.append(up_child)

            down_child = self.move_down()

            if down_child is not None:

                self.children.append(down_child)

            left_child = self.move_left()

            if left_child is not None:

                self.children.append(left_child)

            right_child = self.move_right()

            if right_child is not None:

                self.children.append(right_child)

        return self.children

# Función de Justin 
def writeOutput(Txt):
  
  path_to_goal, cost_of_path, nodes_expande, search_depth, max_search_depth, running_time, max_ram_usage = Txt
  file = open("Output.txt","w")
  file.write(f"""path_to_goal: {path_to_goal}
cost_of_path: {cost_of_path}
nodes_expanded: {nodes_expande}
search_depth: {search_depth}               
max_search_depth: {max_search_depth}
running_time: {"{:.8f}".format(running_time)}
max_ram_usage: {"{:.8f}".format(max_ram_usage)}""")
  file.close()

#Estructura de datos Ricky Montero
class myQueue():
  
    def __init__(self, initial=""):
        self.lista = list()
        self.push(initial)

    def push(self, dato):
        self.lista.append(dato)

    def pop(self):
        dato = self.lista.pop(0)
        return dato

    def top(self):
        dato = self.lista[0]
        return dato

    def empty(self):
        return len(self.lista) == 0

class MyStack:
  
    def __init__(self, initial=""):
        self.lista = list()
        self.push(initial)

    def push(self, dato):
        self.lista.append(dato)

    def pop(self):
       dato = self.lista[-1]
       self.lista.pop()
       return dato

    def top(self):
        dato = self.lista[-1]
        return dato

    def empty(self):
        return len(self.lista) == 0

class MyPriorityQueue:
  
    def __init__(self, initial=""):
        self.lista = []
        self.push(initial)
    
    def push(self, dato):
        self.lista.append(dato)

    def pop(self):
        self.lista = sorted(self.lista, key= lambda x : x[0])
        return self.lista.pop(0)

    def top(self):
        dato = self.lista[0]
        return dato

    def empty(self):
        return len(self.lista) == 0

#Función de Justin
def test_goal(puzzle_state):

    #Indica si se llegó a la meta
    
    puzzle_completed = (0,1,2,3,4,5,6,7,8)
    if puzzle_state.config == puzzle_completed:
      return True


#Calculo de las heuristicas de las posiciones correctas (Albert A)
def calcular_heurisitica(estado):

    #Calcula la distancia Manhattan de una celda
    
    resultado = [0,1,2,3,4,5,6,7,8]
    valor_resultado = 0
    piezas_correctas = 0
    piezas_incorrectas = 0
    for valor_pieza, valor_resultado in zip(list(estado.config), resultado):
        if valor_pieza == valor_resultado:
            piezas_correctas += 1
        else:
            piezas_incorrectas += 1
        valor_resultado += 1 
    return (piezas_incorrectas - 1) + estado.cost


#Funcion que calcula la ruta (Willys Acevedo)
def calcular_ruta(state):

    #Calcula el costo total estimado de un estado Esto es la suma de las distancias Manhattan con respecto a otras celdas
    
    path_to_goal = []
    actual_node = state
    while actual_node.cost >= 1:
        path_to_goal.insert(0, actual_node.action)
        actual_node = actual_node.parent
    return path_to_goal


# Funcion que calcula el costo de una ruta (Willys Acevedo)
def calcular_costo(ruta):
    return len(ruta)



#Algoritmos de busqueda
#Busqueda Bfs de Leonardo Peña
#BFS Search
def bfs_search(initial_state):

    """BFS search"""
    
    frontier = myQueue(initial_state)
    frontier_dic_bfs = {}
    frontier_dic_bfs[tuple(initial_state.config)] = "add"
    explored = set()
    max_search_depth = 0
    nodes_expanded = 0
  
    while not frontier.empty():
        state = frontier.pop()
        explored.add(state.config)

        if test_goal(state):
            path_to_goal = calcular_ruta(state)
            search_depth = len(path_to_goal)            
            return path_to_goal, state.cost, nodes_expanded, search_depth,  max_search_depth

        nodes_expanded += 1

        for node in state.expand():
            if tuple(node.config) not in frontier_dic_bfs and node.config not in explored:
                frontier_dic_bfs[tuple(node.config)] = "add"
                frontier.push(node)
                
                if node.cost > max_search_depth:
                  max_search_depth = node.cost

    return False


# DFS Search
# DFS search de Leonardo Peña
def dfs_search(initial_state):

    """DFS search"""
    frontier = MyStack(initial_state)
    frontier_dic = {}
    frontier_dic[tuple(initial_state.config)] = "add"
    explored = set()
    max_search_depth = 0
    nodes_expanded = 0

    while not frontier.empty():
      state = frontier.pop()
      explored.add(state.config)

      if test_goal(state):
          path_to_goal = calcular_ruta(state)
          search_depth = len(path_to_goal)
          return path_to_goal, state.cost, nodes_expanded, search_depth, max_search_depth
          
      nodes_expanded += 1 
      for node in state.expand()[::-1]:
          if tuple(node.config) not in frontier_dic and node.config not in explored:
              frontier_dic[tuple(node.config)] = "add"
              frontier.push(node)
              if node.cost > max_search_depth:
                  max_search_depth = node.cost
    return False


# A*Search
#A Star Search de Leonardo Peña
def A_star_search(initial_state):

    """A * search"""
    
    ast = Ast((calcular_heurisitica(initial_state), initial_state))
    Ast_dic = {}
    Ast_dic[tuple(initial_state.config)] = "add"
    explored = set()
    max_search_depth = 0
    nodes_expanded = 0
    
    while not ast.empty():  
        state_heuristic, state = ast.pop()
        explored.add(state.config)

        if test_goal(state):
            path_to_goal = calcular_ruta(state)
            search_depth = len(path_to_goal)     
            return path_to_goal, state.cost, nodes_expanded, search_depth, max_search_depth

        nodes_expanded += 1
        for node in state.expand():
            if tuple(node.config) not in Ast_dic and node.config not in explored:
                Ast_dic[tuple(node.config)] = "add"
                ast.push((calcular_heurisitica(node), node))
                if node.cost > max_search_depth:
                  max_search_depth = node.cost
    return False


# Funcion principal llamada desde el punto de acceso Albert
def main():

    query = input().split(" ")

    sm = query[0].lower()

    begin_state = query[1].split(",")

    begin_state = tuple(map(int, begin_state))

    size = int(math.sqrt(len(begin_state)))

    hard_state = PuzzleState(begin_state, size)

    start_time = time()

    if sm == "bfs":

       Txt = bfs_search(hard_state)

    elif sm == "dfs":

       Txt = dfs_search(hard_state)

    elif sm == "ast":

       Txt = A_star_search(hard_state)

    else:

        print("Introduzca comandos de argumentos válidos !")

    path_to_goal, cost_of_path, nodes_expande, search_depth, max_search_depth = Txt

    writeOutput((path_to_goal, cost_of_path, nodes_expande, search_depth, max_search_depth , time() - start_time, memory_usage_psutil()))

if __name__ == '__main__':

    main()

dfs 6,1,8,4,0,2,7,3,5
